In [1]:
import yaml
print("pyyaml is working ✅")


pyyaml is working ✅


In [5]:
import sys
print(sys.executable)


c:\Users\imaya\AppData\Local\Programs\Python\Python312\python.exe


In [4]:
import os

base_dir = os.path.abspath("myfiles/data")
print("Looking in:", base_dir)

if os.path.exists(base_dir):
    for month_folder in os.listdir(base_dir):
        print(month_folder)
else:
    print("Directory does not exist:", base_dir)


Looking in: c:\Users\imaya\Desktop\emayan\myfiles\data
Directory does not exist: c:\Users\imaya\Desktop\emayan\myfiles\data


In [5]:
import os

base_dir = r"C:\Users\imaya\Desktop\data"

print("Looking in:", base_dir)

if os.path.exists(base_dir):
    for month_folder in os.listdir(base_dir):
        print(month_folder)
else:
    print("Directory does not exist:", base_dir)



Looking in: C:\Users\imaya\Desktop\data
2023-10
2023-11
2023-12
2024-01
2024-02
2024-03
2024-04
2024-05
2024-06
2024-07
2024-08
2024-09
2024-10
2024-11


In [2]:
import os
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\imaya\Desktop


In [3]:
import os
os.chdir(r"C:\Users\imaya\Desktop\emayan")


In [12]:
print(df.columns)


Index(['Ticker', 'close', 'date', 'high', 'low', 'month', 'open', 'volume'], dtype='object')


In [17]:
import os

# After calculating gainers, losers, and sector_avg_returns...

output_dir = r"C:\Users\imaya\Desktop\stock_csv"
os.makedirs(output_dir, exist_ok=True)

# 1. Save top gainers per month to CSV
for month, df_gainers in gainers.items():
    file_path = os.path.join(output_dir, f"top_gainers_{month}.csv")
    df_gainers.to_csv(file_path, index=False)

# 2. Save top losers per month to CSV
for month, df_losers in losers.items():
    file_path = os.path.join(output_dir, f"top_losers_{month}.csv")
    df_losers.to_csv(file_path, index=False)

# 3. Save sector average returns to CSV
sector_returns_df = pd.DataFrame(list(sector_avg_returns.items()), columns=['Sector', 'Average_Return'])
sector_returns_df.to_csv(os.path.join(output_dir, "sector_average_returns.csv"), index=False)

print("\n✅ Saved top gainers, losers, and sector returns as CSV files.")



✅ Saved top gainers, losers, and sector returns as CSV files.


In [20]:
from sqlalchemy import create_engine

# Replace with your MySQL username, password, host, port, and database
username = 'root'
password = 'root'
host = 'localhost'
port = '3306'
database = 'stock_csv'

connection_string = f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)


In [21]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to MySQL
engine = create_engine(connection_string)

In [32]:
import os
import pandas as pd

folder_path = r"C:\Users\imaya\Desktop\stock_csv"
files = os.listdir(folder_path)

for file in files:
    file_path = os.path.join(folder_path, file)
    filename, ext = os.path.splitext(file)
    
    if ext.lower() in ['.xls', '.xlsx']:
        try:
            df = pd.read_excel(file_path)
            csv_file_path = os.path.join(folder_path, f"{filename}.csv")
            df.to_csv(csv_file_path, index=False)
            print(f"Converted {file} to {filename}.csv")
        except Exception as e:
            print(f"Failed to convert {file}: {e}")


In [33]:
import os
import yaml
import pandas as pd
import numpy as np
from collections import defaultdict

# --- Sector mapping dictionary (hardcoded) ---
sector_map = {
    "RELIANCE": "Energy",
    "TCS": "Information Technology",
    "HDFC": "Financial Services",
    "INFY": "Information Technology",
    "ICICI": "Financial Services",
    "HDFCBANK": "Financial Services",
    "HINDUNILVR": "Consumer Goods",
    "SBIN": "Financial Services",
    "KOTAKBANK": "Financial Services",
    "ITC": "Consumer Goods",
    "BHARTIARTL": "Telecommunications",
    "LT": "Industrials",
    "ASIANPAINT": "Consumer Goods",
    "BAJAJ-AUTO": "Automobiles",
    "MARUTI": "Automobiles",
    "AXISBANK": "Financial Services",
    "ULTRACEMCO": "Industrials",
    "TITAN": "Consumer Goods",
    "NESTLEIND": "Consumer Goods",
    "M&M": "Automobiles",
    "POWERGRID": "Utilities",
    "INDUSINDBK": "Financial Services",
    "ONGC": "Energy",
    "DRREDDY": "Pharmaceuticals",
    "HCLTECH": "Information Technology",
    "WIPRO": "Information Technology",
    "SBILIFE": "Financial Services",
    "DIVISLAB": "Pharmaceuticals",
    "GRASIM": "Industrials",
    "COALINDIA": "Energy",
    "BAJFINANCE": "Financial Services",
    "EICHERMOT": "Automobiles",
    "TECHM": "Information Technology",
    "JSWSTEEL": "Materials",
    "ADANIPORTS": "Industrials",
    "TATASTEEL": "Materials",
    "BRITANNIA": "Consumer Goods",
    "HINDALCO": "Materials",
    "SHREECEM": "Industrials",
    "HDFCLIFE": "Financial Services",
    "BAJAJFINSV": "Financial Services",
    "NTPC": "Utilities",
    "CIPLA": "Pharmaceuticals",
    "TATAELXSI": "Information Technology",
    "TATAMOTORS": "Automobiles",
    "VEDL": "Materials"
}

# --- 1. Load YAML data and organize per ticker ---
def load_stock_data(base_dir):
    stock_data = defaultdict(list)

    for month_folder in os.listdir(base_dir):
        month_path = os.path.join(base_dir, month_folder)
        if not os.path.isdir(month_path):
            continue

        for filename in os.listdir(month_path):
            if filename.endswith(".yaml"):
                file_path = os.path.join(month_path, filename)
                try:
                    with open(file_path, 'r') as f:
                        records = yaml.safe_load(f)
                        if not records:
                            continue
                        for entry in records:
                            ticker = entry.get("Ticker")
                            if ticker:
                                stock_data[ticker].append(entry)
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")

    return stock_data

# --- 2. Convert stock_data dict to DataFrame per ticker, clean data ---
def create_cleaned_dfs(stock_data):
    ticker_dfs = {}
    for ticker, records in stock_data.items():
        df = pd.DataFrame(records)

        # Ensure date column is datetime
        df['date'] = pd.to_datetime(df['date'])

        # Sort by date ascending
        df.sort_values('date', inplace=True)

        # Remove duplicates
        df.drop_duplicates(inplace=True)

        # Convert numeric columns to numeric types (all lowercase column names)
        for col in ['open', 'close', 'high', 'low', 'volume']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        ticker_dfs[ticker] = df.reset_index(drop=True)
    return ticker_dfs

# --- 3. Calculate yearly return per stock ---
def calculate_yearly_return(df):
    if df.empty:
        return np.nan
    try:
        start_price = df.iloc[0]['open']
        end_price = df.iloc[-1]['close']
        yearly_return = (end_price - start_price) / start_price
        return yearly_return
    except Exception:
        return np.nan

# --- 4. Calculate daily returns and volatility ---
def calculate_daily_returns(df):
    df = df.copy()
    df['daily_return'] = df['close'].pct_change()
    return df

def calculate_volatility(df):
    return df['daily_return'].std()

# --- 5. Calculate cumulative returns ---
def calculate_cumulative_return(df):
    df = df.copy()
    df['cumulative_return'] = (1 + df['daily_return']).cumprod() - 1
    return df

# --- 6. Aggregate yearly return by sector ---
def sector_performance(ticker_returns, sector_map):
    sector_returns = defaultdict(list)
    for ticker, ret in ticker_returns.items():
        sector = sector_map.get(ticker, 'Unknown')  # default to Unknown if ticker not found
        if ret is not None and not np.isnan(ret):
            sector_returns[sector].append(ret)

    sector_avg_returns = {sector: np.nanmean(returns) for sector, returns in sector_returns.items()}
    return sector_avg_returns

# --- 7. Calculate correlation matrix ---
def build_close_price_df(ticker_dfs):
    close_prices = {}
    for ticker, df in ticker_dfs.items():
        close_prices[ticker] = df.set_index('date')['close']
    close_df = pd.DataFrame(close_prices)
    return close_df

# --- 8. Monthly gainers and losers ---
def monthly_gainers_losers(ticker_dfs):
    records = []
    for ticker, df in ticker_dfs.items():
        df = df.copy()
        df['month'] = df['date'].dt.to_period('M')
        df['monthly_return'] = df.groupby('month')['close'].transform(
            lambda x: x.iloc[-1] / x.iloc[0] - 1 if len(x) > 1 else np.nan
        )
        # Get unique month returns
        month_returns = df.drop_duplicates(subset=['month'])[['month', 'monthly_return']]
        for _, row in month_returns.iterrows():
            if pd.notnull(row['monthly_return']):
                records.append({
                    'Ticker': ticker,
                    'Month': str(row['month']),
                    'Monthly_Return': row['monthly_return']
                })
    monthly_df = pd.DataFrame(records)
    return monthly_df

def top_bottom_monthly(monthly_df, top_n=5):
    months = monthly_df['Month'].unique()
    gainers = {}
    losers = {}
    for month in months:
        df_month = monthly_df[monthly_df['Month'] == month]
        gainers[month] = df_month.nlargest(top_n, 'Monthly_Return')
        losers[month] = df_month.nsmallest(top_n, 'Monthly_Return')
    return gainers, losers

# --- Main Execution ---
if __name__ == "__main__":
    BASE_DIR = r"C:\Users\imaya\Desktop\data"   # Change to your actual data folder path

    print("Loading YAML data...")
    stock_data = load_stock_data(BASE_DIR)

    print("Creating cleaned dataframes...")
    ticker_dfs = create_cleaned_dfs(stock_data)

    print("Calculating yearly returns...")
    yearly_returns = {}
    for ticker, df in ticker_dfs.items():
        yearly_returns[ticker] = calculate_yearly_return(df)

    print("Calculating sector performance...")
    sector_avg_returns = sector_performance(yearly_returns, sector_map)

    print("Calculating volatility per stock...")
    volatility = {}
    for ticker, df in ticker_dfs.items():
        df = calculate_daily_returns(df)
        volatility[ticker] = calculate_volatility(df)
        ticker_dfs[ticker] = df  # Update with daily returns and later cumulative return

    print("Calculating cumulative returns...")
    for ticker, df in ticker_dfs.items():
        ticker_dfs[ticker] = calculate_cumulative_return(df)

    print("Building close price dataframe for correlation...")
    close_df = build_close_price_df(ticker_dfs)
    correlation_matrix = close_df.corr()

    print("Calculating monthly gainers and losers...")
    monthly_df = monthly_gainers_losers(ticker_dfs)
    gainers, losers = top_bottom_monthly(monthly_df, top_n=5)

    # Save cleaned CSVs per ticker
    output_dir = r"C:\Users\imaya\Desktop\stock_csv"
    os.makedirs(output_dir, exist_ok=True)

    # 1. Save each ticker cleaned data CSV
    for ticker, df in ticker_dfs.items():
        df.to_csv(os.path.join(output_dir, f"{ticker}.csv"), index=False)

    # 2. Save monthly top gainers CSV files
    for month, df_gainers in gainers.items():
        df_gainers.to_csv(os.path.join(output_dir, f"top_gainers_{month}.csv"), index=False)

    # 3. Save monthly top losers CSV files
    for month, df_losers in losers.items():
        df_losers.to_csv(os.path.join(output_dir, f"top_losers_{month}.csv"), index=False)

    # 4. Save sector average returns CSV
    sector_returns_df = pd.DataFrame(list(sector_avg_returns.items()), columns=['Sector', 'Average_Return'])
    sector_returns_df.to_csv(os.path.join(output_dir, "sector_average_returns.csv"), index=False)

    # Display results
    print("\nTop 10 Yearly Gainers:")
    top_gainers = sorted(yearly_returns.items(), key=lambda x: x[1] if x[1] is not None else -np.inf, reverse=True)[:10]
    for ticker, ret in top_gainers:
        print(f"{ticker}: {ret:.2%}")

    print("\nTop 10 Yearly Losers:")
    top_losers = sorted(yearly_returns.items(), key=lambda x: x[1] if x[1] is not None else np.inf)[:10]
    for ticker, ret in top_losers:
        print(f"{ticker}: {ret:.2%}")

    print("\nSector Average Returns:")
    for sector, ret in sector_avg_returns.items():
        print(f"{sector}: {ret:.2%}")


Loading YAML data...
Creating cleaned dataframes...
Calculating yearly returns...
Calculating sector performance...
Calculating volatility per stock...
Calculating cumulative returns...
Building close price dataframe for correlation...
Calculating monthly gainers and losers...

Top 10 Yearly Gainers:
TRENT: 220.73%
BEL: 103.66%
M&M: 88.64%
BAJAJ-AUTO: 87.20%
BHARTIARTL: 70.90%
POWERGRID: 69.53%
BPCL: 64.05%
HEROMOTOCO: 56.77%
SUNPHARMA: 54.83%
HCLTECH: 54.03%

Top 10 Yearly Losers:
INDUSINDBK: -30.29%
ASIANPAINT: -21.92%
BAJFINANCE: -14.10%
ADANIENT: -7.86%
HINDUNILVR: -1.80%
NESTLEIND: -0.15%
KOTAKBANK: 1.82%
MARUTI: 3.30%
BAJAJFINSV: 4.61%
TITAN: 5.08%

Sector Average Returns:
Financial Services: 5.12%
Consumer Goods: -0.67%
Information Technology: 38.08%
Industrials: 32.17%
Unknown: 61.85%
Energy: 25.92%
Materials: 23.84%
Utilities: 59.55%
Telecommunications: 70.90%
Automobiles: 49.94%
Pharmaceuticals: 17.12%


In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import os

# MySQL credentials and database name
username = 'root'
password = 'root'
host = 'localhost'
port = '3306'
database = 'stock_csv'  # database to create/use

# Folder where your CSV files are stored
csv_folder = r"C:\Users\imaya\Desktop\stock_csv"

# Step 1: Connect to MySQL server (without DB)
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/')

# Step 2: Create database if it doesn't exist
with engine.connect() as conn:
    conn.execute(text(f"CREATE DATABASE IF NOT EXISTS {database}"))
    print(f"Database '{database}' ensured.")

# Step 3: Connect to the new database
engine_db = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

# Step 4: Loop through all CSV files and upload them to MySQL
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        file_path = os.path.join(csv_folder, filename)
        table_name = os.path.splitext(filename)[0]  # file name without .csv
        
        print(f"Uploading {filename} to MySQL table '{table_name}'...")
        
        df = pd.read_csv(file_path)
        
        # Upload to MySQL, replace table if exists
        df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
        
        print(f"Uploaded '{table_name}' successfully.")

print("All CSV files uploaded to MySQL!")



Database 'stock_csv' ensured.
Uploading ADANIENT.csv to MySQL table 'ADANIENT'...
Uploaded 'ADANIENT' successfully.
Uploading ADANIPORTS.csv to MySQL table 'ADANIPORTS'...
Uploaded 'ADANIPORTS' successfully.
Uploading APOLLOHOSP.csv to MySQL table 'APOLLOHOSP'...


C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'ADANIENT' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'ADANIPORTS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'APOLLOHOSP' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='repla

Uploaded 'APOLLOHOSP' successfully.
Uploading ASIANPAINT.csv to MySQL table 'ASIANPAINT'...
Uploaded 'ASIANPAINT' successfully.
Uploading AXISBANK.csv to MySQL table 'AXISBANK'...
Uploaded 'AXISBANK' successfully.
Uploading BAJAJ-AUTO.csv to MySQL table 'BAJAJ-AUTO'...
Uploaded 'BAJAJ-AUTO' successfully.
Uploading BAJAJFINSV.csv to MySQL table 'BAJAJFINSV'...


C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'AXISBANK' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'BAJAJ-AUTO' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'BAJAJFINSV' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='repla

Uploaded 'BAJAJFINSV' successfully.
Uploading BAJFINANCE.csv to MySQL table 'BAJFINANCE'...
Uploaded 'BAJFINANCE' successfully.
Uploading BEL.csv to MySQL table 'BEL'...
Uploaded 'BEL' successfully.
Uploading BHARTIARTL.csv to MySQL table 'BHARTIARTL'...


...

C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'TATACONSUM' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'TATAMOTORS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'TATASTEEL' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='repl

Uploaded 'TATAMOTORS' successfully.
Uploading TATASTEEL.csv to MySQL table 'TATASTEEL'...
Uploaded 'TATASTEEL' successfully.
Uploading TCS.csv to MySQL table 'TCS'...


C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'TCS' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'TECHM' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'TITAN' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=Fals

Uploaded 'TCS' successfully.
Uploading TECHM.csv to MySQL table 'TECHM'...
Uploaded 'TECHM' successfully.
Uploading TITAN.csv to MySQL table 'TITAN'...
Uploaded 'TITAN' successfully.
Uploading top_gainers_2023-10.csv to MySQL table 'top_gainers_2023-10'...
Uploaded 'top_gainers_2023-10' successfully.
Uploading top_gainers_2023-11.csv to MySQL table 'top_gainers_2023-11'...
Uploaded 'top_gainers_2023-11' successfully.
Uploading top_gainers_2023-12.csv to MySQL table 'top_gainers_2023-12'...
Uploaded 'top_gainers_2023-12' successfully.
Uploading top_gainers_2024-01.csv to MySQL table 'top_gainers_2024-01'...
Uploaded 'top_gainers_2024-01' successfully.
Uploading top_gainers_2024-02.csv to MySQL table 'top_gainers_2024-02'...
Uploaded 'top_gainers_2024-02' successfully.
Uploading top_gainers_2024-03.csv to MySQL table 'top_gainers_2024-03'...
Uploaded 'top_gainers_2024-03' successfully.
Uploading top_gainers_2024-04.csv to MySQL table 'top_gainers_2024-04'...
Uploaded 'top_gainers_2024-04

C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'TRENT' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'ULTRACEMCO' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', index=False)
C:\Users\imaya\AppData\Local\Temp\ipykernel_9328\4133618462.py:37: UserWarning: The provided table name 'WIPRO' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(table_name, con=engine_db, if_exists='replace', ind

In [5]:
import os
print(os.getcwd())


C:\Users\imaya\Desktop\emayan


In [7]:
import os
print(os.listdir())


['.env', '.sql', '.venv', 'app.ipynb', 'app.py', 'imdb_2024_combined.csv', 'movie_dashboard.ipynb', 'myenv', 'sector_data.csv', 'stock_dashboard.py', 'venv', '__pycache__']


In [8]:
import pandas as pd

df = pd.read_csv("sector_data.csv")
print(df.head())


             COMPANY         sector                         Symbol
0  ADANI ENTERPRISES  MISCELLANEOUS  ADANI ENTERPRISES: ADANIGREEN
1  ADANI PORTS & SEZ  MISCELLANEOUS  ADANI PORTS & SEZ: ADANIPORTS
2   APOLLO HOSPITALS  MISCELLANEOUS   APOLLO HOSPITALS: APOLLOHOSP
3       ASIAN PAINTS         PAINTS       ASIAN PAINTS: ASIANPAINT
4          AXIS BANK        BANKING            AXIS BANK: AXISBANK


In [11]:
import pandas as pd

# Load the CSV file
sector_df = pd.read_csv("sector_data.csv")

# Extract the ticker after the colon from every row in the 'Symbol' column
sector_df['ticker'] = sector_df['Symbol'].apply(lambda x: x.split(':')[-1].strip())

# Convert ticker to lowercase (to match your MySQL table names)
sector_df['ticker'] = sector_df['ticker'].str.lower()

# Keep only the relevant columns: ticker and sector
sector_df = sector_df[['ticker', 'sector']]

# Display the cleaned dataframe
print(sector_df.tail())   # Or sector_df.to_csv("cleaned_sector_data.csv", index=False) to save


        ticker     sector
45       techm   SOFTWARE
46       titan  RETAILING
47       trent  RETAILING
48  ultracemco     CEMENT
49       wipro   SOFTWARE
